<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>


<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Use Spark and Python to Predict Equipment Purchase</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to get data from the IBM Data Science Experience Community, create a predictive model, and start scoring new data. This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5 and Apache® Spark 2.1.

You will use a publicly available data set, **GoSales Transactions for Naive Bayes Model**, which details anonymous outdoor equipment purchases. Use the details of this data set to predict clients' interests in terms of product line, such as golf accessories, camping equipment, and others.

**Note**: In this notebook we use GOSales data available on [Data Science Experience Community](https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600).

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Set up](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in a Cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  If you do not have existing instance of [Watson Machine Learning (WML) Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) create one (a free plan is offered and information about how to create the instance is [here](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html))
-  Make sure that you are using a Spark 2.1 kernel

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then *read* method. 

In [ ]:
!pip install wget

In [4]:
import wget

link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
filename = wget.download(link_to_data)

print(filename)

GoSales_Tx_NaiveBayes.csv


The csv file GoSales_Tx_NaiveBayes.csv is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [5]:
spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [6]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [7]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [ ]:
print("Number of records: " + str(df_data.count()))

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [17]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))
print("Number of prediction records : " + str(predict_data.count()))

Number of training records: 48176
Number of testing records : 10860
Number of prediction records : 1216


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The predict data set will be used for prediction.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [18]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [19]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [20]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [21]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [22]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [23]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [24]:
train_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



In [25]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [26]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.585359
Test Error = 0.414641


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

First, you must install and import `watson-machine-learning-client` libraries.

**Note**: Python 3.5 and Apache® Spark 2.1 is required.

In [27]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [28]:
!pip install watson-machine-learning-client --upgrade

Requirement already up-to-date: watson-machine-learning-client in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages
Requirement already up-to-date: tqdm in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: tabulate in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: urllib3 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: certifi in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s86b-18e61b28c674e4-3fbaf243aed6/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: pandas in 

In [31]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

Authenticate to Watson Machine Learning service on Cloud.

uthenticate to the Watson Machine Learning service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the [Service Credentials](https://console.bluemix.net/docs/services/service_credentials.html#service_credentials) tab of the service instance that you created on IBM Cloud. 

If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.

In [29]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

In [30]:
# The code was removed by DSX for sharing.

**Tip**: credentials can be found on **Service Credentials** tab of service instance created in Bluemix. If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

In [32]:
client = WatsonMachineLearningAPIClient(wml_credentials)

### 4.1: Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [35]:
published_model_details = client.repository.store_model(model=model_rf, meta_props={'name':'Product line model'}, training_data=train_data, pipeline=pipeline_rf)

In [36]:
model_uid = client.repository.get_model_uid(published_model_details)
print(model_uid)

70a8b8ad-ef04-4d9f-a1e7-679719577b62


Get saved model metadata from Watson Machine Learning.

**Tip**: Use `client.repository.ModelMetaNames.show()` to get the list of available props.

In [37]:
client.repository.ModelMetaNames.show()

-----------------------  ----  --------
META_PROP NAME           TYPE  REQUIRED
NAME                     str   Y
DESCRIPTION              str   N
AUTHOR_NAME              str   N
AUTHOR_EMAIL             str   N
FRAMEWORK_NAME           str   N
FRAMEWORK_VERSION        str   N
FRAMEWORK_LIBRARIES      list  N
RUNTIME_NAME             str   N
RUNTIME_VERSION          str   N
TRAINING_DATA_REFERENCE  dict  N
EVALUATION_METHOD        str   N
EVALUATION_METRICS       list  N
-----------------------  ----  --------


### 4.2: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [38]:
loaded_model = client.repository.load(model_uid)

You can print for example model name to make sure that model has been loaded correctly.

In [39]:
print(type(loaded_model))

<class 'pyspark.ml.pipeline.PipelineModel'>


<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to score test data using loaded model and visualize the prediction results with plotly package.

### 5.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [40]:
predictions = loaded_model.transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [41]:
predictions.show(5)

+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|     PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|PROFESSION|label|PROFESSION_IX|GENDER_IX|MARITAL_STATUS_IX|          features|       rawPrediction|         probability|prediction|      predictedLabel|
+-----------------+------+---+--------------+----------+-----+-------------+---------+-----------------+------------------+--------------------+--------------------+----------+--------------------+
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.40697857263576...|[0.27034892863178...|       1.0|Personal Accessories|
|Camping Equipment|     F| 20|        Single|     Other|  0.0|          0.0|      1.0|              1.0|[1.0,20.0,1.0,0.0]|[5.40697857263576...|[0.27034892863178...|       1.0|Personal Accessories|
|Camping E

By tabulating a count, you can see which product line is the most popular.

In [42]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show(truncate=False)

+------------------------+-----+
|predictedLabel          |count|
+------------------------+-----+
|Camping Equipment       |752  |
|Golf Equipment          |67   |
|Mountaineering Equipment|42   |
|Personal Accessories    |355  |
+------------------------+-----+



### 5.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

First, you need to install required packages. You can do it by running the following code. Run it only one time.

In [ ]:
!pip install plotly
!pip install cufflinks==0.8.2

Import Plotly and other required packages.

In [43]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [44]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

Plot a pie chart that shows the predicted product-line interest.

In [45]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()
product_data = [go.Pie(labels=cumulative_stats.index, values=cumulative_stats['GENDER'])]
product_layout = go.Layout(title='Predicted product line client interest distribution')

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean AGE per product line by using a bar chart.

In [46]:
age_data = [go.Bar(y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"], x=cumulative_stats.index)]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(title = "Product Line", showline=False),
    yaxis=dict(title = "Mean AGE"))

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean age for clients that are interested in golf equipment is predicted to be over 50 years old.

<a id="scoring"></a>
## 6. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record using `watson-machine-learning-client`.

**Note:** You can also use REST API to deploy and score.
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

### 6.1: Create online scoring endpoint

Now you can create an online scoring endpoint. 

#### Create online deployment for published model

In [48]:
deployment_details = client.deployments.create(model_uid=model_uid, name='Product line model deployment')



#######################################################################################

Synchronous deployment creation for uid: '3112f5b0-4c72-4e78-ac33-325b2dc38bbe' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='3112f5b0-4c72-4e78-ac33-325b2dc38bbe'
------------------------------------------------------------------------------------------------




In [49]:
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/3f6e5c2b-4880-46aa-9d79-62e90ccc9d56/published_models/70a8b8ad-ef04-4d9f-a1e7-679719577b62/deployments/3112f5b0-4c72-4e78-ac33-325b2dc38bbe/online


Now, you can send new scoring records (new data) for which you would like to get predictions. To do that, execute the following sample code: 

In [50]:
payload_scoring = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}

client.deployments.score(scoring_url, payload_scoring)

{'fields': ['GENDER',
  'AGE',
  'MARITAL_STATUS',
  'PROFESSION',
  'PRODUCT_LINE',
  'label',
  'PROFESSION_IX',
  'GENDER_IX',
  'MARITAL_STATUS_IX',
  'features',
  'rawPrediction',
  'probability',
  'prediction',
  'predictedLabel'],
 'values': [['M',
   23,
   'Single',
   'Student',
   'Camping Equipment',
   0.0,
   6.0,
   0.0,
   1.0,
   [0.0, 23.0, 1.0, 6.0],
   [5.258712981131878,
    6.723421733525897,
    6.0132018750333724,
    0.218060296213069,
    1.7866031140957863],
   [0.26293564905659383,
    0.3361710866762948,
    0.3006600937516686,
    0.010903014810653448,
    0.0893301557047893],
   1.0,
   'Personal Accessories'],
  ['M',
   55,
   'Single',
   'Executive',
   'Camping Equipment',
   0.0,
   3.0,
   0.0,
   1.0,
   [0.0, 55.0, 1.0, 3.0],
   [3.6000096318286547,
    4.060237825988732,
    3.8628119611335734,
    8.339956386992652,
    0.13698419405638748],
   [0.18000048159143273,
    0.20301189129943661,
    0.19314059805667866,
    0.4169978193496326,
   

As we can see we predict that a 23-year-old male student is interested in personal accessories (predictedLabel: Personal Accessories, prediction: 1.0). We can also see that single, 55 years old man is interested in golf equipment.

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our [Online Documentation](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.